In [8]:

import numpy as np
import matplotlib.pyplot as plt

def syllable_to_phrase_labels(arr, silence=0):
    """
    Convert a sequence of syllable labels into a sequence of phrase labels,
    merging silence bins with their nearest adjacent syllables.


    For each contiguous block of silence:
    - If it's bounded by the same label on both sides, assign that label to all.
    - If it's bounded by two different labels, assign each time-bin to the closer label;
    ties go to the left.
    - If it's at the beginning or end (missing one side), assign to the available side.


    Parameters
    ----------
    arr : np.ndarray
        Array of integer labels, where `silence` frames are indicated by `silence`.
    silence : int, optional
        Integer value representing silence, by default -1.


    Returns
    -------
    np.ndarray
        Array of phrase-level labels with silence frames appropriately merged.
    """
    new_arr = np.array(arr, dtype=int)
    length = len(new_arr)
    if length == 0:
        return new_arr  # Edge case: empty input


    # Helper function to find contiguous regions of silence
    def find_silence_runs(labels):
        runs = []
        in_silence = False
        start = None


        for i, val in enumerate(labels):
            if val == silence and not in_silence:
                in_silence = True
                start = i
            elif val != silence and in_silence:
                runs.append((start, i - 1))
                in_silence = False
        # If ended in silence
        if in_silence:
            runs.append((start, length - 1))
        return runs


    # Identify contiguous silence regions
    silence_runs = find_silence_runs(new_arr)


    for start_idx, end_idx in silence_runs:
        # Check left and right labels
        left_label = new_arr[start_idx - 1] if start_idx > 0 else None
        right_label = new_arr[end_idx + 1] if end_idx < length - 1 else None


        if left_label is None and right_label is None:
            # Entire array is silence or single region with no bounding labels
            # Do nothing or choose some default strategy
            continue
        elif left_label is None:
            # Leading silence; merge with right label
            new_arr[start_idx:end_idx+1] = right_label
        elif right_label is None:
            # Trailing silence; merge with left label
            new_arr[start_idx:end_idx+1] = left_label
        elif left_label == right_label:
            # Same label on both sides
            new_arr[start_idx:end_idx+1] = left_label
        else:
            # Different labels on both sides
            # Assign each bin to whichever side is closer; ties go left
            left_distances = np.arange(start_idx, end_idx + 1) - (start_idx - 1)
            right_distances = (end_idx + 1) - np.arange(start_idx, end_idx + 1)


            for i in range(start_idx, end_idx + 1):
                # Distance from left non-silence is (i - (start_idx - 1))
                dist_left = i - (start_idx - 1)
                # Distance from right non-silence is ((end_idx + 1) - i)
                dist_right = (end_idx + 1) - i


                if dist_left < dist_right:
                    new_arr[i] = left_label
                elif dist_right < dist_left:
                    new_arr[i] = right_label
                else:
                    # Tie -> go left
                    new_arr[i] = left_label


    return new_arr


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
arr   = np.load("/home/george-vengrovski/Documents/projects/tweety_bert_paper/files/llb16_raw_false.npz")
print(arr.files)
X_np  = arr["embedding_outputs"].astype(np.float32)[:50_000]   # slice for demo
y_np  = arr["ground_truth_labels"][:50_000]
N, D  = X_np.shape
print(f"Loaded {N:,}×{D}")
# Visualization parameters
window_size = 1000
step = 100
interval = 30  # ms between frames

# Pick two dimensions for visualization (e.g., first two)
x = X_np[:, 0]
y = X_np[:, 1]
labels = y_np
# Set up the plot
fig, ax = plt.subplots(figsize=(8, 8))
scat_dim = ax.scatter(x, y, c='lightgray', s=1, alpha=0.3, label='All points')
scat_highlight = ax.scatter([], [], c=[], s=8, cmap='tab10', vmin=labels.min(), vmax=labels.max(), label='Active window')
ax.set_title('Scrolling through embedding')
ax.legend()
# Animation update function
def update(frame):
    start = frame * step
    end = min(start + window_size, N)
    x_win = x[start:end]
    y_win = y[start:end]
    labels_win = labels[start:end]
    scat_highlight.set_offsets(np.c_[x_win, y_win])
    scat_highlight.set_array(labels_win)
    return scat_highlight,
num_frames = (N - window_size) // step
from matplotlib.animation import FuncAnimation
ani = FuncAnimation(fig, update, frames=num_frames, interval=interval, blit=True, repeat=True)
# To save as GIF, uncomment the following lines:
ani.save('embedding_scroll.gif', writer='pillow', fps=1000//interval)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# load data
data = np.load("/home/george-vengrovski/Documents/projects/tweety_bert_paper/files/llb16_raw_false.npz")
labels = data["ground_truth_labels"]
embedding = data["embedding_outputs"]

# filter out silence (label 0)
embedding = embedding[labels != 0]

labels = labels[labels != 0]

# compute counts and freqs
unique, counts = np.unique(labels, return_counts=True)
freqs = counts / counts.sum()

# sort descending by freq
order = np.argsort(freqs)[::-1]
labels_sorted = unique[order]
freqs_sorted = freqs[order]

# plot at consecutive x positions
x = np.arange(len(labels_sorted))
plt.figure()
plt.bar(x, freqs_sorted)
plt.xticks(x, labels_sorted)
plt.xlabel("label")
plt.ylabel("frequency")
plt.title("llb16 label frequencies (desc sorted)")
plt.tight_layout()
plt.show()



In [ ]:
import matplotlib.pyplot as plt

# assume `labels` is shape (n_samples,) and `embedding` is shape (n_samples, 2)
x = embedding[:, 0]
y = embedding[:, 1]

plt.figure(figsize=(6, 6))
scatter = plt.scatter(x, y, c=labels, cmap='tab20', s=5)
plt.colorbar(scatter, label='phrase label')
plt.xlabel('embedding dim 0')
plt.ylabel('embedding dim 1')
plt.title('embedding colored by phrase labels')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1) load data
data = np.load("/home/george-vengrovski/Documents/projects/tweety_bert_paper/files/llb16_raw_false.npz")
labels    = data["ground_truth_labels"]
embedding = data["embedding_outputs"]
specs     = data["s"]  # shape: (n_timebins, n_freq_bins)

# 2) remove silence if desired
mask = labels != 0
labels    = labels[mask]
embedding = embedding[mask]
specs     = specs[mask]

# 3) define embedding-space fence
x1, x2 = 10, 20
y1, y2 = -10, -1
x = embedding[:, 0]
y = embedding[:, 1]
region_mask = (x > x1) & (x < x2) & (y > y1) & (y < y2)

# 4) subset to region
labels_r    = labels[region_mask]
specs_r     = specs[region_mask]    # (m_timebins, n_freq_bins)
embedding_r = embedding[region_mask]

# 5) find contiguous runs of the same label within the region
segments = []
if len(labels_r) > 0:
    start = 0
    current = labels_r[0]
    for i, lab in enumerate(labels_r[1:], start=1):
        if lab != current:
            segments.append((current, start, i))
            current = lab
            start = i
    segments.append((current, start, len(labels_r)))

# 6) plot up to the first 10 segments as spectrograms
for idx, (lab, start, end) in enumerate(segments[:10], 1):
    seg = specs_r[start:end]  # shape = (timebins, freq_bins)
    plt.figure(figsize=(8, 3))
    plt.imshow(seg.T, aspect="auto", origin="lower")
    plt.title(f"segment {idx}: label={lab}, bins={start}→{end-1} (N={end-start})")
    plt.xlabel("timebin")
    plt.ylabel("freq bin")
    plt.colorbar(label="amplitude")
    plt.tight_layout()
    plt.show()




In [ ]:
llb16_layer0_attention_output.npz
llb16_layer0_feed_forward_output.npz
llb16_layer0_feed_forward_output_gelu.npz
llb16_layer0_intermediate_residual_stream.npz
llb16_layer1_attention_output.npz
llb16_layer1_feed_forward_output.npz
llb16_layer1_feed_forward_output_gelu.npz
llb16_layer1_intermediate_residual_stream.npz
llb16_layer2_attention_output.npz
llb16_layer2_feed_forward_output.npz
llb16_layer2_feed_forward_output_gelu.npz
llb16_layer2_intermediate_residual_stream.npz
llb16_layer3_attention_output.npz
llb16_layer3_feed_forward_output.npz
llb16_layer3_feed_forward_output_gelu.npz
llb16_layer3_intermediate_residual_stream.npz